In [2]:
%run "../00_project_config.ipynb"
%run "data_objects.ipynb"

import pandas as pd
import pathlib as pl
import sys

sample_annotation = PROJECT_BASE.joinpath(
    "annotations", "projectmng", "hgsvc_samples.tsv"
)

data_status_annotation = PROJECT_BASE.joinpath(
    "annotations", "projectmng", "data_production_status.tsv"
)

samples_df = pd.read_csv(sample_annotation, sep="\t", header=0, comment="#")

data_status_df = pd.read_csv(data_status_annotation, sep="\t", header=0, comment="#")

DATA_STATUS = dict()
for row in data_status_df.itertuples(index=False):
    if row.sample == "GM19320":
        sys.stderr.write("\nSkipping over 19320 - not enough data\n")
        continue
    if row.hifi_complete == "skip" or row.ont_complete == "skip" or row.igsr_status == "skip":
        continue
    if row.hifi_complete == "no" or row.ont_complete == "no":
        assert row.igsr_status == "incomplete", row
    if row.hifi_complete == "yes" and row.ont_complete == "yes":
        assert row.igsr_status == "complete"
    sample_name = row.sample
    if sample_name.startswith("GM"):
        sample_name = "NA" + sample_name[2:]
    sample_info = {
        "order_num": int(row.order_num),
        "sex": row.sex.lower(),
        "hifi_complete": True if row.hifi_complete == "yes" else False,
        "ont_complete": True if row.ont_complete == "yes" else False
    }
    DATA_STATUS[sample_name] = sample_info

SAMPLES = dict()
for row in samples_df.itertuples(index=False):
    if row.sample == "GM19320":
        sys.stderr.write("\nSkipping over 19320 - not enough data\n")
        continue
    sample = Sample(row.sample, row.order_num, row.sex)
    try:
        sample_data_status = DATA_STATUS[sample.name]
        assert sample_data_status["order_num"] == sample.hgsvc_num
        assert sample_data_status["sex"] == sample.sex
        sample.hifi_complete = sample_data_status["hifi_complete"]
        sample.ont_complete = sample_data_status["ont_complete"]
    except KeyError:
        pass
    SAMPLES[sample.name] = sample
    SAMPLES[sample.alt] = sample


ADD_SAMPLES = [
    ("NA24143", 100, "female"),
    ("NA24149", 101, "male")
]

for sample_name, num, sex in ADD_SAMPLES:
    sample = Sample(sample_name, num, sex)
    SAMPLES[sample.name] = sample
    SAMPLES[sample.alt] = sample
    


1: NA19238 (HiFi complete: False / ONT complete: False)
2: NA19239 (HiFi complete: False / ONT complete: True)
3: NA19240 (HiFi complete: False / ONT complete: False)
4: HG00731 (HiFi complete: False / ONT complete: False)
5: HG00732 (HiFi complete: True / ONT complete: True)
6: HG00733 (HiFi complete: True / ONT complete: True)
7: HG00512 (HiFi complete: True / ONT complete: False)
8: HG00513 (HiFi complete: True / ONT complete: False)
9: HG00514 (HiFi complete: True / ONT complete: False)
10: HG03732 (HiFi complete: True / ONT complete: False)
11: HG01573 (HiFi complete: True / ONT complete: True)
12: HG00171 (HiFi complete: True / ONT complete: True)
13: HG02587 (HiFi complete: True / ONT complete: True)
14: HG01114 (HiFi complete: True / ONT complete: True)
15: HG02018 (HiFi complete: True / ONT complete: True)
16: NA19036 (HiFi complete: True / ONT complete: True)
17: NA12329 (HiFi complete: True / ONT complete: False)
18: HG02011 (HiFi complete: True / ONT complete: False)
19: NA


Skipping over 19320 - not enough data

Skipping over 19320 - not enough data
